In [26]:
# Import libraries
import pandas as pd
import pandas.io.sql as sqlio
import sqlalchemy
import pprint
import numpy as np
import psycopg2
import matplotlib.pyplot as plt
%matplotlib inline
from sqlalchemy import create_engine

In [31]:
# Use a txt file with the db info that has been .gitignored
text = open("connection.txt", "r").read()

# Use psycopg2 for connection because sqlalchemy was giving me headaches
conn = psycopg2.connect(text)

# Get the three tables to do analysis for and test to make sure the data is coming through
employees_data = pd.read_sql("Select * FROM employees;", conn)
salaries_data = pd.read_sql("Select * FROM salaries;", conn)
titles_data = pd.read_sql("Select * FROM titles;", conn)
salaries_data[0:10]

,emp_no,salary
0,10001,60117
1,10002,65828
2,10003,40006
3,10004,40054
4,10005,78228
5,10006,40000
6,10007,56724
7,10008,46671
8,10009,60929
9,10010,72488
